In [6]:
import sys
print(sys.version)

3.5.2 (default, Aug 18 2017, 17:48:00) 
[GCC 5.4.0 20160609]


In [7]:
#! pip3 install pandas

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# train = pd.read_csv('datasets/train.csv')

# # train.head
# train.iloc[:10,:10]
# N, _ = train.shape

# for i in range(3):
#     file = "datasets/train_batch_{}".format(i)
#     with open(file,'wb') as f:
#         pickle.dump(train.iloc[int(2e5)*i:min(N+1,int(2e5) * (i+1)),:] ,file = f)
    
# test = pd.read_csv('datasets/test.csv')
# test.shape

# N, _ = test.shape

# for i in range(5):
#     file = "datasets/test_batch_{}".format(i)
#     with open(file,'wb') as f:
#         pickle.dump(test.iloc[int(2e5)*i:min(N+1,int(2e5) * (i+1)),:] ,file = f)

In [62]:
import data_util
import data_preprocess
import datetime

tic= datetime.datetime.now()
train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

naive_pre = data_preprocess.naive_preprocess()
train_data = naive_pre.dtype_change(train_data)
#train_prop is specified
X_train, X_val, y_train, y_val = data_util.split_train(train_data,prop=0.75)

X_train = naive_pre.scale(X_train)
X_val = naive_pre.scale(X_val,test=True)

In [54]:
print(X_train.shape ,X_val.shape)
# dev_train.dtypes
# dev_train.head
# train_data.isnull().any()

(446409, 57) (148803, 57)


In [28]:
from sklearn.neural_network import MLPClassifier

hidden_layer_size = [(50,30)]
learning_rate = [0.001]
reg = []
momentum =[0.9]
rso= np.random.RandomState(66)

#for now I don't mess up with inits
mlp = MLPClassifier(hidden_layer_sizes= hidden_layer_size[0],random_state=rso)
mlp.fit(X_train,y_train)

In [ ]:
#check the performance
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
test_predictions = mlp.predict_proba(dev_train)
roc_auc_score(dev_y,test_predictions)

In [69]:
# sum(test_predictions)
# sum(dev_y)
#test_predictions
sum(test_predictions[:,0])
roc_auc_score(dev_y,test_predictions[:,1])

1.0